In [1]:
import ee
import geemap
import folium
import pandas as pd

In [2]:
ee.Authenticate()
ee.Initialize(project='gee-map-id')

In [3]:
pip install dash dash-bootstrap-components


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'd:\AgriMatrix\Area Calculation and Classification\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [5]:
import ee
import geemap
import pandas as pd
import plotly.express as px
import dash
from dash import html, dcc, Input, Output
import dash_bootstrap_components as dbc
from dash.dependencies import State
import base64
import io


# Load Sri Lanka districts from FAO GAUL dataset
districts = ee.FeatureCollection("FAO/GAUL/2015/level1") \
    .filter(ee.Filter.eq("ADM0_NAME", "Sri Lanka"))

# Load Sentinel-2 ImageCollection
image = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterBounds(districts.geometry())
    .filterDate("2021-01-01", "2021-12-31")
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 10))
    .median()
    .clip(districts)
    .select(["B2", "B3", "B4", "B8"])
)

# Define land cover classes
landcover_palette = ["006400", "FFA500", "808080"]
landcover_names = ["Vegetation", "Flat Lands", "Urban"]

# Training data
vegetation = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Point([80.7698, 7.8731]), {"landcover": 0}),
    ee.Feature(ee.Geometry.Point([80.7748, 7.8781]), {"landcover": 0}),
])

flatlands = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Point([80.7715, 7.8700]), {"landcover": 1}),
    ee.Feature(ee.Geometry.Point([80.7750, 7.8650]), {"landcover": 1}),
])

urban = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Point([80.7800, 7.8800]), {"landcover": 2}),
    ee.Feature(ee.Geometry.Point([80.7840, 7.8830]), {"landcover": 2}),
])

training_points = vegetation.merge(flatlands).merge(urban)
bands = ["B2", "B3", "B4", "B8"]

# Train and classify
training = image.select(bands).sampleRegions(
    collection=training_points,
    properties=["landcover"],
    scale=10
)

classifier = ee.Classifier.smileRandomForest(50).train(
    features=training,
    classProperty="landcover",
    inputProperties=bands
)

classified = image.select(bands).classify(classifier)

# Calculate land cover areas per district
def calculate_areas(fc):
    district_stats = []

    def compute_area(feature):
        district_name = feature.get("ADM1_NAME")
        stats = classified.reduceRegion(
            reducer=ee.Reducer.frequencyHistogram(),
            geometry=feature.geometry(),
            scale=30,
            maxPixels=1e13
        )
        hist = stats.get("classification")
        return ee.Feature(None, {
            "district": district_name,
            "hist": hist
        })

    results = districts.map(compute_area).getInfo()["features"]

    for f in results:
        props = f["properties"]
        name = props["district"]
        hist = props.get("hist", {})
        row = {"District": name}
        total = sum(hist.values())
        for i in range(3):
            value = hist.get(str(i), 0)
            row[landcover_names[i]] = round(value * 30 * 30 / 10000, 2)  # hectares
        district_stats.append(row)

    return pd.DataFrame(district_stats)

# Prepare data
df = calculate_areas(districts)

# Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = "Sri Lanka Landcover Analysis"

app.layout = html.Div([
    html.H2("District-wise Landcover Analysis of Sri Lanka", className="text-center mt-4"),
    dbc.Container([
        dcc.Dropdown(
            id="district-dropdown",
            options=[{"label": d, "value": d} for d in df["District"]],
            placeholder="Select a district...",
            className="mb-3"
        ),
        dbc.Row([
            dbc.Col(dcc.Graph(id="bar-chart"), width=6),
            dbc.Col(dcc.Graph(id="pie-chart"), width=6),
        ]),
        html.Br(),
        html.H4("Landcover Area Table (hectares)"),
        dbc.Table.from_dataframe(df, striped=True, bordered=True, hover=True),
        html.Br(),
        html.Button("Download CSV", id="btn_csv", className="btn btn-success"),
        dcc.Download(id="download-dataframe-csv"),
    ])
])

@app.callback(
    Output("bar-chart", "figure"),
    Output("pie-chart", "figure"),
    Input("district-dropdown", "value")
)
def update_graphs(selected_district):
    if not selected_district:
        filtered = df.mean(numeric_only=True)
        title = "Average Landcover Distribution"
    else:
        filtered = df[df["District"] == selected_district].iloc[0]
        title = f"{selected_district} - Landcover Distribution"

    values = [filtered.get(k, 0) for k in landcover_names]
    fig_bar = px.bar(
        x=landcover_names,
        y=values,
        labels={"x": "Landcover Type", "y": "Area (ha)"},
        title=title,
        color=landcover_names
    )

    fig_pie = px.pie(
        names=landcover_names,
        values=values,
        title=title
    )

    return fig_bar, fig_pie

@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("btn_csv", "n_clicks"),
    prevent_initial_call=True
)
def download_csv(n):
    return dcc.send_data_frame(df.to_csv, "sri_lanka_landcover_districts.csv", index=False)

# Run the app
#if __name__ == "__main__":
 #   app.run(debug=False)

if __name__ == "__main__":
    app.run(debug=False, port=8051)


